In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#from src.data.data_collector import DataCollector
from src.models.model_trainer_rl_v2_2_buyhold import ModelTrainerRL, TradingEnvRL
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from src.models.backtester import PortfolioBacktester, PortfolioBacktesterRL
from src.utils.config_loader import load_config


config = load_config("config/config.yaml")

d:\MSDS\buy-sell-hold-strategy-prediction\venv-3.11\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
d:\MSDS\buy-sell-hold-strategy-prediction\venv-3.11\Lib\site-packages\kaleido\scopes\plotly.py:32: DeprecationWarning:


Use of plotly.io.kaleido.scope.default_format is deprecated and support will be removed after September 2025.
Please use plotly.io.defaults.default_format instead.


d:\MSDS\buy-sell-hold-strategy-prediction\venv-3.11\Lib\site-packages\kaleido\scopes\plotly.py:33: DeprecationWarning:




## V2.2 PPO Agent

### Function

In [ ]:
def run_ppo_trading_pipeline(stock_symbol, config, year='2022', save_path="models/", show_plot=True):
    """
    Complete PPO trading pipeline: load data, train model, generate predictions, and backtest.
    
    Parameters:
    -----------
    stock_symbol : str
        Stock ticker symbol (e.g., 'CNP', 'MDU')
    config : dict
        Configuration dictionary loaded from config.yaml
    year : str, optional
        Year for data file selection (default: '2022')
    save_path : str, optional
        Directory to save/load models (default: 'models/')
    show_plot : bool, optional
        Whether to display the portfolio plot (default: True)
    
    Returns:
    --------
    tuple : (portfolio, metrics, actions)
        - portfolio: VectorBT portfolio object
        - metrics: Dictionary of performance metrics
        - actions: Array of predicted actions
    """
    # 1. Load Data
    try:
        data = pd.read_csv(f'data/processed/{stock_symbol}_processed_{year}.csv')
        if 'Date' in data.columns:
            data['Date'] = pd.to_datetime(data['Date'])
            data.set_index('Date', inplace=True)
        print(f"Data loaded successfully for {stock_symbol}.")
    except FileNotFoundError:
        print(f"Error: Data file not found for {stock_symbol}. Check path.")
        sys.exit()
    
    # 2. Split Train/Test
    split_idx = int(len(data) * 0.7)
    train_df = data.iloc[:split_idx]
    test_df = data.iloc[split_idx:]
    
    # 3. Training Phase
    print(f"Training PPO Agent for {stock_symbol}...")
    trainer = ModelTrainerRL(config['reinforcement_learning'])
    env_params = config['reinforcement_learning']['environment']
    
    env_train = TradingEnvRL(
        train_df, 
        initial_balance=env_params.get('initial_balance', 10000),
        commission=env_params.get('commission', 0.001),
        lookback_window=env_params.get('lookback_window', 30),
        reward_func='profit'
    )
    
    result = trainer.train_ppo(env_train)
    trainer.save_models(save_path)
    print("Training Complete. Models saved.")
    
    # 4. Inference Phase
    print("Generating Agent Predictions on Test Data...")
    model = PPO.load(os.path.join(save_path, "ppo_model"))
    
    env_test = TradingEnvRL(
        test_df, 
        initial_balance=env_params.get('initial_balance', 100000),
        commission=env_params.get('commission', 0.001),
        lookback_window=env_params.get('lookback_window', 30),
        reward_func='profit' 
    )
    
    vec_env_test = DummyVecEnv([lambda: env_test])
    
    norm_path = os.path.join(save_path, "ppo_vecnormalize.pkl")
    if os.path.exists(norm_path):
        vec_env_test = VecNormalize.load(norm_path, vec_env_test)
        vec_env_test.training = False
        vec_env_test.norm_reward = False
    else:
        print("WARNING: Normalization stats not found. Model predictions may be garbage.")
    
    obs = vec_env_test.reset()
    done = [False]
    actions = []
    
    while not done[0]:
        action, _ = model.predict(obs, deterministic=True)
        actions.append(action[0])
        obs, _, done, _ = vec_env_test.step(action)
    
    print(f"Generated {len(actions)} actions.")
    
    # 5. Backtesting Phase
    print("Running Backtest...")
    backtester = PortfolioBacktesterRL(env_params)
    
    portfolio = backtester.run_backtest(
        price_data=test_df['close'], 
        predicted_weights=np.array(actions).flatten(),
        lookback_window=env_params.get('lookback_window', 30)
    )
    
    comparison = backtester.compare_with_buy_and_hold_rl()
    metrics = backtester.get_performance_metrics()
    
    print(f"\n--- Strategy Performance for {stock_symbol} ---")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")
    
    if show_plot:
        portfolio.plot().show()
        # Print trade statistics
        trades = portfolio.trades.records_readable
        print(f"\n--- Trade Statistics for {stock_symbol} ---")
        print(f"Total number of trades: {len(trades)}")
        print("\nTrade Direction Counts:")
        print(trades['Direction'].value_counts())

        # Analyze trade outcomes
        if 'PnL' in trades.columns:
                profitable_trades = trades[trades['PnL'] > 0]
                loss_trades = trades[trades['PnL'] < 0]
                
                print(f"\n--- Trade Outcomes ---")
                print(f"Number of profitable exits: {len(profitable_trades)}")
                print(f"Number of loss exits (cut loss): {len(loss_trades)}")
                print(f"Win rate: {len(profitable_trades) / len(trades) * 100:.2f}%")
                print(f"Average profit per winning trade: ${profitable_trades['PnL'].mean():.2f}" if len(profitable_trades) > 0 else "No profitable trades")
                print(f"Average loss per losing trade: ${loss_trades['PnL'].mean():.2f}" if len(loss_trades) > 0 else "No losing trades")
            


    
    return portfolio, metrics, np.array(actions).flatten()

In [3]:
# ## Single stock
# stock_symbol = "CNP"
# portfolio, metrics, actions = run_ppo_trading_pipeline(stock_symbol, config)

In [4]:
# # portfolio.trades.records_readable
# # Print trade statistics
# trades = portfolio.trades.records_readable
# print(f"\n--- Trade Statistics for abc ---")
# print(f"Total number of trades: {len(trades)}")
# print("\nTrade Direction Counts:")
# print(trades['Direction'].value_counts())

# # Analyze trade outcomes
# if 'PnL' in trades.columns:
#         profitable_trades = trades[trades['PnL'] > 0]
#         loss_trades = trades[trades['PnL'] < 0]
        
#         print(f"\n--- Trade Outcomes ---")
#         print(f"Number of profitable exits: {len(profitable_trades)}")
#         print(f"Number of loss exits (cut loss): {len(loss_trades)}")
#         print(f"Win rate: {len(profitable_trades) / len(trades) * 100:.2f}%")
#         print(f"Average profit per winning trade: ${profitable_trades['PnL'].mean():.2f}" if len(profitable_trades) > 0 else "No profitable trades")
#         print(f"Average loss per losing trade: ${loss_trades['PnL'].mean():.2f}" if len(loss_trades) > 0 else "No losing trades")
    


In [6]:
# Multiple stocks in a loop
stocks = ["AAPL", "AMZN", "TSLA", "BAC","MDU", "CWCO", "NEE", "DUK"]
results = {}

for stock in stocks:
    print(f"\n{'='*60}")
    print(f"Processing {stock}")
    print(f"{'='*60}")
    portfolio, metrics, actions = run_ppo_trading_pipeline(
        stock_symbol=stock, 
        config=config,
        show_plot=True  # Don't show plots in loop
    )
    results[stock] = {'portfolio': portfolio, 'metrics': metrics, 'actions': actions}

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_36288\1970358808.py:29: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v2_2_buyhold:Training PPO...



Processing AAPL
Data loaded successfully for AAPL.
Training PPO Agent for AAPL...
Using cpu device
-----------------------------
| time/              |      |
|    fps             | 2381 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 2037         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0040051187 |
|    clip_fraction        | 0.0213       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.42        |
|    explained_variance   | 0.00198      |
|    learning_rate        | 0.0002       |
|    loss                 | 0.0664       |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00313     |
| 

INFO:src.models.model_trainer_rl_v2_2_buyhold:Training PPO for 200000 timesteps
INFO:BacktesterRL:Preparing Backtest. Raw Prices: 279, Predictions: 218
INFO:BacktesterRL:Running vectorbt simulation...
INFO:BacktesterRL:Backtest successfully completed.


Saved model and normalization stats for models/ppo_vecnormalize.pkl
Training Complete. Models saved.
Generating Agent Predictions on Test Data...
Generated 218 actions.
Running Backtest...


INFO:BacktesterRL:Strategy Return: 93.71%
INFO:BacktesterRL:Buy & Hold Return: 12.81%
INFO:BacktesterRL:Outperformance: 80.91%


...

INFO:src.models.model_trainer_rl_v2_2_buyhold:Training PPO for 200000 timesteps
INFO:BacktesterRL:Preparing Backtest. Raw Prices: 279, Predictions: 218
INFO:BacktesterRL:Running vectorbt simulation...
INFO:BacktesterRL:Backtest successfully completed.


Saved model and normalization stats for models/ppo_vecnormalize.pkl
Training Complete. Models saved.
Generating Agent Predictions on Test Data...
Generated 218 actions.
Running Backtest...


INFO:BacktesterRL:Strategy Return: 37.71%
INFO:BacktesterRL:Buy & Hold Return: 19.81%
INFO:BacktesterRL:Outperformance: 17.90%



--- Strategy Performance for DUK ---
Total Return (%): 37.7100
Annual Return (%): 70.8700
Sharpe Ratio: 3.1346
Sortino Ratio: 5.1131
Max Drawdown (%): -6.6900
Calmar Ratio: 10.5851
Win Rate (%): 68.7500
Total Trades: 32.0000
Final Value ($): 137707.5300



--- Trade Statistics for DUK ---
Total number of trades: 32

--- Trade Outcomes ---
Number of profitable exits: 22
Number of loss exits (cut loss): 10
Win rate: 68.75%
Average profit per winning trade: $2164.10
Average loss per losing trade: $-990.27
